In [ ]:
import keras
from keras import layers
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.python.framework import ops


from keras.utils import load_img
from keras.utils import array_to_img
from keras.utils import img_to_array
from keras.preprocessing import image_dataset_from_directory
import tensorflow as tf

import os
import math
import numpy as np

from IPython.display import display

In [ ]:
! gdown --id 1AacCu4C_rfOtm-LINJd-KByt_r_PhREi



In [ ]:
!unzip '/content/DIV2K_train_LR_bicubic_X2 (2).zip' -d '/content/extracted_data_train/'

In [ ]:
! gdown --id 1sHbd5Tm_wjFPwXHShoQLzS2RedJFd4_9

In [ ]:
!unzip '/content/DIV2K_test_LR_bicubic_x2.zip' -d '/content/extracted_data_test/'

In [ ]:
! gdown --id 1y2GbjCpxLe574r0BnEvDvlrTrnFOZ27E

In [ ]:
!unzip '/content/DIV2K_valid_LR_bicubic_X2.zip' -d '/content/extracted_data_valid/'

In [ ]:
SUPER_RES_MODEL = '/content/super_res_model'
TRAINING_PLOT =  '/content/training_plot'
VISUALIZATION_PATH = '/content/visualizations'


os.makedirs(SUPER_RES_MODEL, exist_ok=True)
os.makedirs(TRAINING_PLOT, exist_ok=True)
os.makedirs(VISUALIZATION_PATH, exist_ok=True)



In [ ]:

ORIG_SIZE = (300, 300)
DOWN_FACTOR = 6

RDB_LAYERS = 3
BATCH_SIZE = 8
EPOCHS = 20
LR = 1e-3


In [ ]:


import tensorflow as tf
def process_input(imagePath, downFactor=DOWN_FACTOR):

	resizeShape = ORIG_SIZE[0] // downFactor

	origImage = tf.io.read_file(imagePath)
	origImage = tf.image.decode_png(origImage, 3)
	origImage = tf.image.convert_image_dtype(origImage, tf.float32)
	origImage = tf.image.resize(origImage, ORIG_SIZE,
		method="area")


	origImageYUV = tf.image.rgb_to_yuv(origImage)
	(target, _, _) = tf.split(origImageYUV, 3, axis=-1)

	downImage = tf.image.resize(target, [resizeShape, resizeShape],
		method="area")

	target = tf.clip_by_value(target, 0.0, 1.0)
	downImage = tf.clip_by_value(downImage, 0.0, 1.0)

	return (downImage, target)

In [ ]:
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import tensorflow as tf
def rdb_block(inputs, numLayers):

	channels = inputs.get_shape()[-1]
	storedOutputs = [inputs]

	for _ in range(numLayers):

		localConcat = tf.concat(storedOutputs, axis=-1)
		out = Conv2D(filters=channels, kernel_size=3, padding="same",
			activation="relu",
			kernel_initializer="Orthogonal")(localConcat)
		storedOutputs.append(out)

	finalConcat = tf.concat(storedOutputs, axis=-1)
	finalOut = Conv2D(filters=inputs.get_shape()[-1], kernel_size=1,
		padding="same", activation="relu",
		kernel_initializer="Orthogonal")(finalConcat)
	finalOut = Add()([finalOut, inputs])

	return finalOut

In [ ]:
def get_subpixel_net(downsampleFactor=DOWN_FACTOR, channels=1, rdbLayers=RDB_LAYERS):

    inputs = Input((None, None, 1))

    x = Conv2D(64, 5, padding="same", strides=(1, 1), activation="relu", kernel_initializer="Orthogonal")(inputs)
    x = Conv2D(64, 3, padding="same", strides=(1, 1), activation="relu", kernel_initializer="Orthogonal")(x)



    x = Conv2D(32, 3, padding="same", activation="relu", strides=(1, 1), kernel_initializer="Orthogonal")(x)
    x = Conv2D(32, 3, padding="same", activation="relu", strides=(1, 1), kernel_initializer="Orthogonal")(x)


    x = rdb_block(x, numLayers=rdbLayers)
    x = Conv2D(32, 3, padding="same", strides=(1, 1), activation="relu", kernel_initializer="Orthogonal")(x)
    x = rdb_block(x, numLayers=rdbLayers)
    x = Conv2D(16, 2, padding="same", strides=(1, 1), activation="relu", kernel_initializer="Orthogonal")(x)
    x = rdb_block(x, numLayers=rdbLayers)





    # x = Conv2D(256, 3, padding="same", activation="relu", strides=(1, 1), kernel_initializer="Orthogonal")(x)

    x = Conv2D(channels * (downsampleFactor ** 2), 3, padding="same",
               activation="sigmoid", kernel_initializer="Orthogonal")(x)

    # x = Conv2D(36, kernel_size=3, padding='same', activation='sigmoid')(x)

    outputs = tf.nn.depth_to_space(x, downsampleFactor)



    model = Model(inputs, outputs)


    return model



In [ ]:
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow as tf

def psnr(orig, pred):

	orig = orig * 255.0
	orig = tf.cast(orig, tf.uint8)
	orig = tf.clip_by_value(orig, 0, 255)

	pred = pred * 255.0
	pred = tf.cast(pred, tf.uint8)
	pred = tf.clip_by_value(pred, 0, 255)

	return tf.image.psnr(orig, pred, max_val=255)



AUTO = tf.data.AUTOTUNE

print("[INFO] loading images from disk...")
trainPaths = list(paths.list_images('/content/extracted_data_train'))
valPaths = list(paths.list_images('/content/extracted_data_valid'))
trainDS = tf.data.Dataset.from_tensor_slices(trainPaths)
valDS = tf.data.Dataset.from_tensor_slices(valPaths)



In [ ]:

from tensorflow.keras.layers import BatchNormalization


trainDS = trainDS.map(process_input,
					  num_parallel_calls=AUTO).batch(
	BATCH_SIZE).prefetch(AUTO)
valDS = valDS.map(process_input,
				  num_parallel_calls=AUTO).batch(
	BATCH_SIZE).prefetch(AUTO)

print("[INFO] initializing and training model...")
model = get_subpixel_net()
model.summary()
model.compile(optimizer="adam", loss="mse", metrics=psnr)
H = model.fit(trainDS, validation_data=valDS, epochs=EPOCHS)


plt.style.use("ggplot")
plt.figure()
plt.plot(H.history["loss"], label="train_loss")
plt.plot(H.history["val_loss"], label="val_loss")
plt.plot(H.history["psnr"], label="train_psnr")
plt.plot(H.history["val_psnr"], label="val_psnr")
plt.title("Training Loss and PSNR")
plt.xlabel("Epoch #")
plt.ylabel("Loss/PSNR")
plt.legend(loc="lower left")
plt.savefig(TRAINING_PLOT)

print("[INFO] serializing model...")
model.save(SUPER_RES_MODEL)

In [ ]:
def psnr(orig, pred):

	orig = orig * 255.0
	orig = tf.cast(orig, tf.uint8)
	orig = tf.clip_by_value(orig, 0, 255)

	pred = pred * 255.0
	pred = tf.cast(pred, tf.uint8)
	pred = tf.clip_by_value(pred, 0, 255)

	return tf.image.psnr(orig, pred, max_val=255)

In [ ]:

def load_image(imagePath):

	orig = load_img(imagePath)
	downsampled = orig.resize((orig.size[0] // DOWN_FACTOR,
		orig.size[1] // DOWN_FACTOR), Image.BICUBIC)

	return (orig, downsampled)

In [ ]:
def get_y_channel(image):

	ycbcr = image.convert("YCbCr")
	(y, cb, cr) = ycbcr.split()

	y = np.array(y)
	y = y.astype("float32") / 255.0

	return (y, cb, cr)

In [ ]:
def clip_numpy(image):

	image = tf.cast(image * 255.0, tf.uint8)
	image = tf.clip_by_value(image, 0, 255).numpy()

	return image

In [ ]:
def postprocess_image(y, cb, cr):

	y = clip_numpy(y).squeeze()
	y = y.reshape(y.shape[0], y.shape[1])
	y = Image.fromarray(y, mode="L")

	outputCB= cb.resize(y.size, Image.BICUBIC)
	outputCR= cr.resize(y.size, Image.BICUBIC)

	final = Image.merge("YCbCr", (y, outputCB, outputCR)).convert("RGB")
	return np.array(final)

In [ ]:

from tensorflow.keras.models import load_model

print("[INFO] loading test images...")
testPaths = list(paths.list_images('/content/extracted_data_test'))
currentTestPaths = np.random.choice(testPaths, 10)
# currentTestPaths = '/content/extracted_data_test/DIV2K_test_LR_bicubic_x2/0793x2.png'

print("[INFO] loading model...")
superResModel = load_model(SUPER_RES_MODEL,
	custom_objects={"psnr" : psnr})

In [ ]:
from PIL import Image


print("[INFO] performing predictions...")
for (i, path) in enumerate(currentTestPaths):

    (orig, downsampled) = load_image(path)


    (y, cb, cr) = get_y_channel(downsampled)
    upscaledY = superResModel.predict(y[None, ...])[0]

    finalOutput = postprocess_image(upscaledY, cb, cr)



    path = os.path.join(VISUALIZATION_PATH, f"{i}_viz.png")
    (fig, ((ax1, ax4, ax2))) = plt.subplots(nrows=1, ncols=3, figsize=(18, 18))

    ax1.imshow(downsampled)
    ax1.set_title("Preprocessed Downscaled Image")

    ax2.imshow(orig)
    ax2.set_title("Original High-Res Image")



    ax4.imshow(finalOutput)
    ax4.set_title("Super-res Model")

    fig.savefig(path, dpi=300, bbox_inches="tight")
    plt.show()

